In [1]:
# mpnn design all the looped poses

from glob import glob

with open("03_pair_looped/03_paired_looped_poses.list", 'w') as f:
    for fname in sorted(glob("/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/03_pair_looped/decoys/*/*.pdb.bz2")):
        f.write(fname + "\n")

In [ ]:
import sys
sys.path.insert(0, '/home/broerman/crispy_shifty/')

from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "04_two_state"
design_list_file = "/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/03_pair_looped/03_paired_looped_poses.list"
output_path = f"/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/04_two_state/{simulation_name}"

options = " ".join(
    [
        "out:level 200",
    ]
)

extra_kwargs = {
    "num_sequences": "10",
    "batch_size" = "10",
    "mpnn_temperature": 0.2,
    "mpnn_design_area": "scan",
    "mpnn_betas": "scan",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_dimers",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="12G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)